In [1]:
import xarray as xr
import numpy as np
import pandas as pd

In [14]:
def rsds_to_cf(rsds_bc, ref_mon):
    # Valor máximo de irradiancia como normalizador (por celda)
    rsds_max = np.nanpercentile(ref_mon.values, 99, axis=0)
    rsds_max = np.where(np.isnan(rsds_max), 1.0, rsds_max)  # evita división NaN

    cf = rsds_bc / rsds_max
    cf = cf.clip(0, 1)  # limitar a rango válido
    return cf

In [ ]:
def export_cf(cf_regional, model, ssp, outdir):
    df = cf_regional.to_pandas()

    # convertir índice cftime a strings YYYY-MM-01
    df.index = [f"{t.year:04d}-{t.month:02d}-01" for t in df.index]
    df.index.name = "timesteps"
    
    out_path = f"{outdir}/pv_cf_{model}_{ssp}.csv"
    df.to_csv(out_path, float_format="%.3f")
    print(f"[OK] Guardado {out_path}")
    return out_path


In [7]:
ZARR_PATH = "/home/aninotna/magister/tesis/justh2_pipeline/data/solar/solar_diario_grilla.zarr"  # referencia diaria (ghi)

In [4]:
def std_names(ds):
    rename = {}
    if "latitude" in ds.coords and "lat" not in ds.coords: rename["latitude"]="lat"
    if "longitude" in ds.coords and "lon" not in ds.coords: rename["longitude"]="lon"
    return ds.rename(rename) if rename else ds

In [5]:
def ensure_lat_ascending(ds):
    if "lat" in ds.coords and ds.lat.ndim==1 and ds.lat.values[0] > ds.lat.values[-1]:
        ds = ds.sortby("lat")
    return ds

In [31]:
# Abrir zarr (usa dask por defecto)
ref = xr.open_zarr(ZARR_PATH)
ref = std_names(ref)
ref = ensure_lat_ascending(ref)

# Variable de referencia
ref_var = "ghi"
ref_da = ref[ref_var]  # (date, lat, lon)

# Diario -> Mensual (promedio mensual de W/m^2 diarios)
ref_mon = ref_da.resample(date="MS").mean("date")  # (time≈'date', lat, lon)
ref_mon = ref_mon.rename({"date":"time"})
ref_mon = ref_mon.assign_coords(time=ref_mon["time"].dt.floor("D"))

# Años comunes esperados (2004–2014)
ref_mon = ref_mon.sel(time=slice("2004-01-01","2014-12-31"))


In [32]:
outdir = "/home/aninotna/magister/tesis/justh2_pipeline/data/pv_cf"

for model in ["access_cm2", "gfdl_esm4"]:
    for ssp in ["ssp245", "ssp370", "ssp585"]:
        fn = f"/home/aninotna/magister/tesis/justh2_pipeline/data/cmip6/bias_corrected/rsds_bc_Amon_{model}_{ssp}.nc"
        ds = xr.open_dataset(fn)
        rsds_bc = ds["rsds_bc"]

        cf = rsds_to_cf(rsds_bc, ref_mon)
        cf_regional = cf.mean(dim=["lat","lon"])
        export_cf(cf_regional, model, ssp, outdir)


/home/aninotna/.conda/envs/pv-cf-cmip6/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1620: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


[OK] Guardado /home/aninotna/magister/tesis/justh2_pipeline/data/pv_cf/pv_cf_access_cm2_ssp245.csv


/home/aninotna/.conda/envs/pv-cf-cmip6/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1620: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


[OK] Guardado /home/aninotna/magister/tesis/justh2_pipeline/data/pv_cf/pv_cf_access_cm2_ssp370.csv


/home/aninotna/.conda/envs/pv-cf-cmip6/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1620: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


...

/home/aninotna/.conda/envs/pv-cf-cmip6/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1620: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


[OK] Guardado /home/aninotna/magister/tesis/justh2_pipeline/data/pv_cf/pv_cf_gfdl_esm4_ssp245.csv


/home/aninotna/.conda/envs/pv-cf-cmip6/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1620: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


[OK] Guardado /home/aninotna/magister/tesis/justh2_pipeline/data/pv_cf/pv_cf_gfdl_esm4_ssp370.csv
[OK] Guardado /home/aninotna/magister/tesis/justh2_pipeline/data/pv_cf/pv_cf_gfdl_esm4_ssp585.csv


/home/aninotna/.conda/envs/pv-cf-cmip6/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1620: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,
